本文档使用了Python的离散事件仿真库对于排队论模型进行了仿真  
仿真的主要目的是提供个性化定制，如对分布的设定，对排队规则的设定等。  
本文档提供了： 
基础排队模型仿真  
Erlang分布实现  
通用分布函数适配器  
如有额外需求，可以前往SimPy官网进行查阅
Copy Right：@MCM 2019 SJTU 61组 朱文杰

# SimPy 核心概念
https://zhuanlan.zhihu.com/p/31526894<br/>
SimPy 是离散事件驱动的仿真库。<br/>
所有活动部件，例如车辆、顾客,、即便是信息，都可以用 process (进程) 来模拟。<br/>
这些 process 存放在 environment (环境) 。所有 process 之间，以及与environment 之间的互动，通过 event (事件) 来进行.<br/>

process 表达为 generators (生成器)， 构建event(事件)并通过 yield 语句抛出事件。<br/>

当一个进程抛出事件，进程会被暂停，直到事件被激活(triggered)。多个进程可以等待同一个事件。 SimPy 会按照这些进程抛出的事件激活的先后， 来恢复进程。<br/>

其实中最重要的一类事件是 Timeout， 这类事件允许一段时间后再被激活， 用来表达一个进程休眠或者保持当前的状态持续指定的一段时间。这类事件通过 Environment.timeout来调用。<br/>

Environment<br/>

Environment 决定仿真的起点/终点， 管理仿真元素之间的关联, 主要 API 有<br/>

simpy.Environment.process - 添加仿真进程<br/>
simpy.Environment.event - 创建事件<br/>
simpy.Environment.timeout - 提供延时(timeout)事件<br/>
simpy.Environment.until - 仿真结束的条件（时间或事件）<br/>
simpy.Environment.run - 仿真启动<br/>

Resource 和 Store

Resource/Store 也是另外一类重要的核心概念, 但凡仿真中涉及的人力资源以及工艺上的物料消耗都会抽象用 Resource 来表达, 主要的 method 是 request. Store 处理各种优先级的队列问题, 表现跟 queue 一致, 通过 method get / put 存放 item<br/>

Store - 抽象队列<br/>

simpy.Store - 存取 item 遵循仿真时间上的先到后到<br/>
simpy.PriorityStore - 存取 item 遵循仿真时间上的先到后到同时考虑人为添加的优先级<br/>
simpy.FilterStore - 存取 item 遵循仿真时间上的先到后到, 同时队列中存在分类, 按照不同类别进行存取<br/>
simpy.Container - 表达连续/不可分的物质, 包括液体/气体的存放, 存取的是一个 float 数值<br/>
Resource - 抽象资源<br/>

simpy.Resource - 表达人力资源或某种限制条件, 例如某个工序可调用的工人数, 可以调用的机器数<br/>
simpy.PriorityResource - 兼容Resource的功能, 添加可以插队的功能, 高优先级的进程可以优先调用资源, 但只能是在前一个被服务的进程结束以后进行插队<br/>
simpy.PreemptiveResource - 兼容Resource的功能, 添加可以插队的功能, 高优先级的进程可以打断正在被服务的进程进行插队<br/>

In [1]:
from numpy.random import *
from simpy import *

高阶函数随机数生成器  

输入：分布函数，单一参数(多参数请柯里化)  
输出：随机数

In [2]:
def rng(dis,param):
    """random number generator"""
    def generate():
        return dis(lam=param,size=1)[0]
    return generate

Erlang分布函数：  
输入：结束  
输出：k阶erlang分布函数

In [3]:
def erlang(k):
    """由k个指数分布拟合"""
    def exp2erlang(lam,size):
        res=[]
        for n in range(size):
            k_poisson= exponential(lam/k,size=k)
            sum=0
            for x in k_poisson:
                sum = sum + x
            res.append(sum)
        return res
    return exp2erlang

In [4]:
#测试，计算分布期望
x=rng(erlang(10),10)
sum=0
for i in range(10000):
    sum= sum+x()
print(sum/10000)

9.99565983119657


In [5]:
#典型银行模型：FIFO
def bankSample(X,Y,Z,A,B,EX):
    """
银行排队服务例子

情景:
  一个柜台对客户进行服务, 服务耗时, 客户等候过长会离开柜台
    %X 表示时间间隔分布
    %Y 表示服务时间的分布
    %Z 表示服务台的个数
    %A 表示系统的容量,此处特殊化为客户的耐心时间分布
    %B 表示顾客数
    %以上参数必须有界，受到计算机精度限制，可以使用大常数近似无穷
    %C 表示服务规则,请修改函数
    %EX 传递了银行储蓄额的分布
  """
    #加入随机种子是为了对比模型的变化
    seed(2)
    def source(env, number, interval,counter):
        """生成客户"""
        for i in range(number):
            c = customer(env, '客户%04d' % i, counter, time_in_bank=Y(),account=EX())
            env.process(c)
            yield env.timeout(interval)
    #成功服务的客户
    SUCC=0
    #成功客户等待时间
    WAIT=0
    #成功客户逗留时间
    STAY=0
    #业务额
    AMT=0
    def customer(env, name, counter, time_in_bank,account):
        nonlocal WAIT
        nonlocal SUCC
        nonlocal STAY
        nonlocal AMT
        """顾客服务与离开仿真"""
        arrive = env.now
        #print('%7.4f  %s: 到达' % (arrive, name))
        with counter.request() as req:
            patience = A()
            # 直到到达或者失去耐心
            results = yield req | env.timeout(patience)
            wait = env.now - arrive
        
            if req in results:
                # 到达
                WAIT=WAIT+wait
                STAY=STAY+time_in_bank
                AMT= AMT + account
                #print('%7.4f %s:等待%6.3f' % (env.now, name, wait))
                yield env.timeout(time_in_bank)
                SUCC=SUCC+1
                #print('%7.4f %s:服务完成' % (env.now, name))
            else:
                # We reneged
                pass
                #print('%7.4f %s:等待%6.3f后离开' % (env.now, name, wait))
    # 初始化环境
    print('排队问题仿真')
    env = Environment()

    # 开始协程
    counter = Resource(env, capacity=Z)
    env.process(source(env, B, X(), counter))
    env.run()
    print("总服务人数：{0:n}人".format(SUCC))
    print("总营业额：{0:n}元".format(AMT))
    print("总计失去： {0:n}名客户".format(B-SUCC))
    print("损失率为： {0:n}%".format((B-SUCC)/B*100))
    print("平均等待时间：{0:n}".format(WAIT/SUCC) )
    print("平均耗费时间：{0:n}".format(STAY/SUCC) )

In [6]:
#间隔分布
X=rng(erlang(3),3)
#服务时间分布
Y=rng(erlang(3),10)
#耐心时间分布
A=rng(erlang(3),3)
#业务额分布
def normaltocurry(s):
    def normalcurry(lam,size):
        return normal(lam,s,size=size)
    return normalcurry
EX=rng(normaltocurry(200),1000)

In [7]:
bankSample(X,Y,3,A,1000,EX)

排队问题仿真
总服务人数：417人
总营业额：409903元
总计失去： 583名客户
损失率为： 58.3%
平均等待时间：1.8828
平均耗费时间：10.0787


In [8]:
#银行模型·ELite：优先队列
def eliteBankSample(X,Y,Z,A,B,EX):
    """
银行排队服务例子

情景:
  一个柜台对客户进行服务, 服务耗时, 客户等候过长会离开柜台
    %X 表示时间间隔分布
    %Y 表示服务时间的分布
    %Z 表示服务台的个数
    %A 表示系统的容量,此处特殊化为客户的耐心时间分布
    %B 表示顾客数
    %以上参数必须有界，受到计算机精度限制，可以使用大常数近似无穷
    %C 表示服务规则,请修改函数
    %EX 传递了银行储蓄额的分布
  """
    #加入随机种子是为了对比模型的变化
    seed(2)
    def source(env, number, interval,counter):
        """生成客户"""
        for i in range(number):
            c = customer(env, '客户%04d' % i, counter, time_in_bank=Y(),account=EX())
            env.process(c)
            yield env.timeout(interval)
    #成功服务的客户
    SUCC=0
    #成功客户等待时间
    WAIT=0
    #成功客户逗留时间
    STAY=0
    #业务额
    AMT=0
    def customer(env, name, counter, time_in_bank,account):
        nonlocal WAIT
        nonlocal SUCC
        nonlocal STAY
        nonlocal AMT
        """顾客服务与离开仿真"""
        arrive = env.now
        #print('%7.4f  %s: 到达' % (arrive, name))
        #以业绩作为优先级，priority越小，优先级越大，
        with counter.request(priority = 1/account) as req:
            patience = A()
            # 直到到达或者失去耐心
            results = yield req | env.timeout(patience)
            wait = env.now - arrive
        
            if req in results:
                # 到达
                WAIT=WAIT+wait
                STAY=STAY+time_in_bank
                AMT= AMT + account
                #print('%7.4f %s:等待%6.3f' % (env.now, name, wait))
                yield env.timeout(time_in_bank)
                SUCC=SUCC+1
                #print('%7.4f %s:服务完成' % (env.now, name))
            else:
                # We reneged
                pass
                #print('%7.4f %s:等待%6.3f后离开' % (env.now, name, wait))
    # 初始化环境
    print('排队问题仿真')
    env = Environment()

    # 开始协程
    counter = PriorityResource(env, capacity=Z)
    env.process(source(env, B, X(), counter))
    env.run()
    print("总服务人数：{0:n}人".format(SUCC))
    print("总营业额：{0:n}元".format(AMT))
    print("总计失去： {0:n}名客户".format(B-SUCC))
    print("损失率为： {0:n}%".format((B-SUCC)/B*100))
    print("平均等待时间：{0:n}".format(WAIT/SUCC) )
    print("平均耗费时间：{0:n}".format(STAY/SUCC) )

In [9]:
#间隔分布
X=rng(erlang(3),3)
#服务时间分布
Y=rng(erlang(3),10)
#耐心时间分布
A=rng(erlang(3),3)
#业务额分布
def normaltocurry(s):
    def normalcurry(lam,size):
        return normal(lam,s,size=size)
    return normalcurry
EX=rng(normaltocurry(200),1000)

In [10]:
eliteBankSample(X,Y,3,A,1000,EX)

排队问题仿真
总服务人数：432人
总营业额：450145元
总计失去： 568名客户
损失率为： 56.8%
平均等待时间：1.28862
平均耗费时间：9.69799
